In [5]:
!pip install -U langchain-google-genai pillow

import os
import glob
import base64
import io
from PIL import Image
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=GOOGLE_API_KEY,
    temperature=0
)

print("✅ set up perfectly!")

✅ set up perfectly!


In [ ]:
def image_to_base64(pil_image):
    """
    将 PIL 图片转换为 Base64 编码字符串，防止报错
    """
    buffered = io.BytesIO()
    pil_image.convert('RGB').save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/jpeg;base64,{img_str}"

def load_bill_images(folder_path="/content/bills"):
    """
    读取指定文件夹下的所有账单图片
    """

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"⚠️ 文件夹 {folder_path} 不存在，已自动创建。请上传图片到这里！")
        return []

    image_paths = glob.glob(os.path.join(folder_path, "*"))

    valid_exts = ['.jpg', '.jpeg', '.png', '.webp']
    image_paths = [p for p in image_paths if os.path.splitext(p)[1].lower() in valid_exts]

    images = []
    print(f"📂 正在读取 '{folder_path}' ...")

    for path in image_paths:
        try:
            img = Image.open(path)
            images.append(img)
            print(f"  - 已加载: {os.path.basename(path)}")
        except Exception as e:
            print(f"  ⚠️ 无法读取: {path} ({e})")

    print(f"✅ 共成功加载 {len(images)} 张图片")
    return images

current_images = load_bill_images()

📂 正在读取 '/content/bills' ...
  - 已加载: 8af49969-7257-475d-862a-5fffa9254f84.jpg
  - 已加载: 96b9cada-591b-4972-820b-ba27d00e5e73.jpg
  - 已加载: 905b22cb-1213-4b3f-bed0-8c7d55dab836.jpg
  - 已加载: 34aa8044-346a-4df0-a18a-596405f3f168.jpg
  - 已加载: aa3c8ae2-50b4-4ea4-9c5e-db46ae99d43b.jpg
  - 已加载: 945281b9-dcf8-4dc5-976d-ee86c4b950d4.jpg
  - 已加载: 37e58d8f-890f-4cc7-8263-3e0e40e72d8a.jpg
✅ 共成功加载 7 张图片


In [7]:
def process_query(user_query, image_folder="/content/bills"):
    images = load_bill_images(image_folder)

    if not images:
        return "❌ 错误：没有找到图片。请先上传图片到 /content/bills 文件夹。"

    prompt_text = f"""
    You are a financial assistant tailored for analyzing supermarket bills.
    I am providing you with {len(images)} image(s) of bills.

    The user query is: "{user_query}"

    Please execute the following logic STRICTLY:

    1. **Target: Total Spending**
       - If asking for total money spent: Extract "Total" or "Payment" from EACH bill and sum them up.
       - Return the calculation and final result with currency.

    2. **Target: Original Price (Without Discount)**
       - If asking for cost without discount: Identify "Subtotal" (before savings) OR add "Savings" back to "Total" for EACH bill.
       - Sum these original amounts up.
       - Return the calculation and final result.

    3. **Target: Irrelevant Queries**
       - If query is unrelated to bills/shopping (e.g., "President of USA", "Capital of France"), REJECT it.
       - Reply exactly: "Irrelevant query."

    Provide a clear, step-by-step response.
    """

    content_parts = [{"type": "text", "text": prompt_text}]

    for img in images:
        base64_str = image_to_base64(img)
        content_parts.append({"type": "image_url", "image_url": base64_str})

    message = HumanMessage(content=content_parts)

    try:
        print(f"🤖 AI 正在思考: '{user_query}' ...")
        response = llm.invoke([message])
        return response.content
    except Exception as e:
        return f"❌ 调用出错: {e}"

In [8]:
print("="*20 + " 测试开始 " + "="*20)

q1 = "How much money did I spend in total for these bills?"
print(f"\n📝 Q1: {q1}")
print("-" * 40)
print(process_query(q1))

q2 = "How much would I have had to pay without the discount?"
print(f"\n📝 Q2: {q2}")
print("-" * 40)
print(process_query(q2))

q3 = "Who is the president of USA?"
print(f"\n📝 Q3: {q3}")
print("-" * 40)
print(process_query(q3))

print("\n" + "="*20 + " 测试结束 " + "="*20)

==================== 测试开始 ====================

📝 Q1: How much money did I spend in total for these bills?
----------------------------------------
📂 正在读取 '/content/bills' ...
  - 已加载: 8af49969-7257-475d-862a-5fffa9254f84.jpg
  - 已加载: 96b9cada-591b-4972-820b-ba27d00e5e73.jpg
  - 已加载: 905b22cb-1213-4b3f-bed0-8c7d55dab836.jpg
  - 已加载: 34aa8044-346a-4df0-a18a-596405f3f168.jpg
  - 已加载: aa3c8ae2-50b4-4ea4-9c5e-db46ae99d43b.jpg
  - 已加载: 945281b9-dcf8-4dc5-976d-ee86c4b950d4.jpg
  - 已加载: 37e58d8f-890f-4cc7-8263-3e0e40e72d8a.jpg
✅ 共成功加载 7 张图片
🤖 AI 正在思考: 'How much money did I spend in total for these bills?' ...
Here's the breakdown of your spending based on the provided bills:

**1. Total Spending**

*   **Bill 1:** $394.70
*   **Bill 2:** $514.00
*   **Bill 3:** $102.30
*   **Bill 4:** $316.11
*   **Bill 5:** $315.64
*   **Bill 6:** $190.80
*   **Bill 7:** $140.80

**Calculation:** $394.70 + $514.00 + $102.30 + $316.11 + $315.64 + $190.80 + $140.80 = $1974.35

**Total Spending:** $1974.35

📝 Q